# ETAPA 1

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.master("local[*]").appName("Exercicio intro").getOrCreate()

In [3]:
df_nomes = spark.read.csv('../exercicio_1/nomes_aleatorios.txt')

In [4]:
df_nomes.show(5)

+-----------------+
|              _c0|
+-----------------+
|       Edna Riina|
|Theresa Samaniego|
|        Paul Cook|
|  Michael Darnell|
|     Martha Smith|
+-----------------+
only showing top 5 rows


# ETAPA 2

In [5]:
df_nomes.printSchema()
df_nomes = df_nomes.withColumnRenamed("_c0",'Nomes')
df_nomes.printSchema()
df_nomes.show(10)

root
 |-- _c0: string (nullable = true)

root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|       Edna Riina|
|Theresa Samaniego|
|        Paul Cook|
|  Michael Darnell|
|     Martha Smith|
|      Alan Rorick|
|       John Smith|
|       Carol Holt|
|   Gloria Beckham|
|     Bart Aguilar|
+-----------------+
only showing top 10 rows


# ETAPA 3

In [6]:
df_nomes = df_nomes.withColumn("aleatorio", F.rand())

df_nomes = df_nomes.withColumn("Escolaridade", 
    F.when(F.col("aleatorio") < 0.33, "Fundamental")
   .when(F.col("aleatorio") < 0.66, "Medio")
   .otherwise("Superior")
)

df_nomes = df_nomes.drop("aleatorio")

In [7]:
df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|       Edna Riina|    Superior|
|Theresa Samaniego| Fundamental|
|        Paul Cook|    Superior|
|  Michael Darnell|       Medio|
|     Martha Smith|    Superior|
|      Alan Rorick|    Superior|
|       John Smith|    Superior|
|       Carol Holt|    Superior|
|   Gloria Beckham| Fundamental|
|     Bart Aguilar|       Medio|
+-----------------+------------+
only showing top 10 rows


# ETAPA 4

In [8]:
paises = [
    'Argentina', 'Bolívia', 'Brasil', 'Chile', 'Colômbia', 'Equador', 'Guiana', 'França (Guiana Francesa)', 
    'Paraguai', 'Peru', 'Suriname', 'Uruguai', 'Venezuela'
]

In [9]:
df_nomes = df_nomes.withColumn("Pais", F.expr(
    f"element_at(array({', '.join([repr(pais) for pais in paises])}), cast(rand() * {len(paises)} + 1 as int))"
))

df_nomes.show(5)

+-----------------+------------+--------------------+
|            Nomes|Escolaridade|                Pais|
+-----------------+------------+--------------------+
|       Edna Riina|    Superior|França (Guiana Fr...|
|Theresa Samaniego| Fundamental|            Colômbia|
|        Paul Cook|    Superior|           Venezuela|
|  Michael Darnell|       Medio|             Uruguai|
|     Martha Smith|    Superior|           Argentina|
+-----------------+------------+--------------------+
only showing top 5 rows


# ETAPA 5

In [10]:
df_nomes = df_nomes.withColumn("AnoNascimento", F.floor(F.rand() * (2010 - 1945 + 1)) + 1945)
df_nomes.show(5)

+-----------------+------------+--------------------+-------------+
|            Nomes|Escolaridade|                Pais|AnoNascimento|
+-----------------+------------+--------------------+-------------+
|       Edna Riina|    Superior|França (Guiana Fr...|         1960|
|Theresa Samaniego| Fundamental|            Colômbia|         1975|
|        Paul Cook|    Superior|           Venezuela|         1966|
|  Michael Darnell|       Medio|             Uruguai|         1979|
|     Martha Smith|    Superior|           Argentina|         1985|
+-----------------+------------+--------------------+-------------+
only showing top 5 rows


# ETAPA 6

In [11]:
df_select = df_nomes.select("Nomes").where("AnoNascimento >= 2001")
df_select.show(10)

+----------------+
|           Nomes|
+----------------+
|    Bart Aguilar|
|  Lois Bernstein|
|      Glenn Mack|
|     Erma Rideau|
| Patrick Johnson|
|  David Sullivan|
|   Cindy Ramirez|
|   Vance Melcher|
|Karen Altamirano|
|  Ronald Johnson|
+----------------+
only showing top 10 rows


# ETAPA 7

In [12]:
df_nomes.createOrReplaceTempView("pessoas")

df_select = spark.sql("""
    SELECT Nomes
    FROM pessoas
    WHERE AnoNascimento >= 2001
""")

df_select.show(10)

+----------------+
|           Nomes|
+----------------+
|    Bart Aguilar|
|  Lois Bernstein|
|      Glenn Mack|
|     Erma Rideau|
| Patrick Johnson|
|  David Sullivan|
|   Cindy Ramirez|
|   Vance Melcher|
|Karen Altamirano|
|  Ronald Johnson|
+----------------+
only showing top 10 rows


# ETAPA 8

In [13]:
print(df_nomes.filter(
    (F.col("AnoNascimento") >= 1980) & (F.col("AnoNascimento") <= 1994)
).count())

2272851


# ETAPA 9

In [14]:
resultado = spark.sql("""
    SELECT COUNT(*) AS total_millennials
    FROM pessoas
    WHERE AnoNascimento BETWEEN 1980 AND 1994
""")

resultado.show()

+-----------------+
|total_millennials|
+-----------------+
|          2272851|
+-----------------+



# ETAPA 10

In [15]:
df_geracoes = spark.sql("""
    SELECT
        Pais,
        CASE
            WHEN AnoNascimento BETWEEN 1944 AND 1964 THEN 'Baby Boomers'
            WHEN AnoNascimento BETWEEN 1965 AND 1979 THEN 'Geração X'
            WHEN AnoNascimento BETWEEN 1980 AND 1994 THEN 'Millennials'
            WHEN AnoNascimento BETWEEN 1995 AND 2015 THEN 'Geração Z'
            ELSE 'Desconhecida'
        END AS Geracao,
        COUNT(*) AS total
    FROM pessoas
    GROUP BY Pais, Geracao
    ORDER BY Pais ASC, Geracao ASC, total ASC
""")

In [16]:
df_geracoes.show()

+---------+------------+------+
|     Pais|     Geracao| total|
+---------+------------+------+
|Argentina|Baby Boomers|234029|
|Argentina|   Geração X|174455|
|Argentina|   Geração Z|187073|
|Argentina| Millennials|174761|
|  Bolívia|Baby Boomers|232865|
|  Bolívia|   Geração X|174378|
|  Bolívia|   Geração Z|186757|
|  Bolívia| Millennials|174862|
|   Brasil|Baby Boomers|233503|
|   Brasil|   Geração X|174778|
|   Brasil|   Geração Z|186909|
|   Brasil| Millennials|174789|
|    Chile|Baby Boomers|233138|
|    Chile|   Geração X|174132|
|    Chile|   Geração Z|186513|
|    Chile| Millennials|174837|
| Colômbia|Baby Boomers|233457|
| Colômbia|   Geração X|174663|
| Colômbia|   Geração Z|187319|
| Colômbia| Millennials|174991|
+---------+------------+------+
only showing top 20 rows
